# PROJECT 3 :

### Author : Omer Ozeren 

## Introduction

Using any of the three classifiers described in chapter 6 of Natural Language Processing with Python, and any features you can think of, build the best name gender classifier you can. Begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the dev-test set, and the remaining 6900 words for the training set. Then, starting with the example name gender classifier, make incremental improvements. Use the dev-test set to check your progress. Once you are satisfied with your classifier, check its final performance on the test set. How does the performance on the test set compare to the performance on the dev-test set? Is this what you'd expect?

#### Import Packages

In [1]:
import nltk
from sklearn.metrics import confusion_matrix, classification_report
from nltk.corpus import names
import pandas as pd
import random
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
nltk.download('names')


Bad key "text.kerning_factor" on line 4 in
/Users/omerozeren/opt/anaconda3/envs/sps620env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
[nltk_data] Downloading package names to
[nltk_data]     /Users/omerozeren/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

I am going to use Naive Bayes as a Classifier to build a name gender classifier.I am going to use several different feature engineering process to select feasible features to be implemented in Naive Bayes model.In addition ,i will compare Naive Bayes models with different features with accuracy.

### Data Collection

In [2]:
# load names
labeled_names = ([(name, "male") for name in names.words("male.txt")] + 
                 [(name, "female") for name in names.words("female.txt")])
# random shuffle the names
random.shuffle(labeled_names)

I will begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the dev-test set, and the remaining 6900 words for the training set. 

In [3]:
# split data into three subsets
data_train = labeled_names[1500:]
data_devtest = labeled_names[500:1000]
data_test = labeled_names[:500]

**I also need to create a function to count error which presents variance between actual and model prediction.Please click [here](http://www.nltk.org/book/ch06.html) for references.**

In [4]:
# define error analysis function
def error_analysis(gender_features,classifier,display=False,dataset='devtest'):
    # error list
    errors = []
    data = data_devtest if dataset =='devtest' else data_test
    # extract mislabels
    for (name, tag) in data:
        guess = classifier.classify(gender_features(name))
        if guess != tag:
            errors.append((tag, guess, name))
    if display :        
       print("Number of Errors: ", len(errors))
       # print the errors
       for (tag, guess, name) in sorted(errors):
         print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))
    return len(errors)    

### Generating New Features

I will  generate the new features to Which is using first letter and last letter to review  performance in datasets.
Please click [here](http://www.nltk.org/book/ch06.html)  for references.

#### First Letter function :

In [5]:
def first_letter(name):
    features = {}
    features["first_letter"] = name[0].lower()
    return features

In [6]:
train_dataset_first_letter = [(first_letter(n), g) for (n,g) in data_train]
devtest_dataset_first_letter = [(first_letter(n), g) for (n,g) in data_devtest]
test_dataset_first_letter = [(first_letter(n), g) for (n,g) in data_test]
nb1 = nltk.NaiveBayesClassifier.train(train_dataset_first_letter) 

print('Dev-set accuracy is')
print(nltk.classify.accuracy(nb1, devtest_dataset_first_letter))
print('Test accuracy is')
print(nltk.classify.accuracy(nb1, test_dataset_first_letter))

Dev-set accuracy is
0.638
Test accuracy is
0.67


In [7]:
error_analysis(first_letter,nb1,display=True)

Number of Errors:  181
correct=female   guess=male     name=Hedy                          
correct=female   guess=male     name=Hestia                        
correct=female   guess=male     name=Hetti                         
correct=female   guess=male     name=Hildagarde                    
correct=female   guess=male     name=Honoria                       
correct=female   guess=male     name=Hyacinthie                    
correct=female   guess=male     name=Hynda                         
correct=female   guess=male     name=Wandie                        
correct=female   guess=male     name=Wendy                         
correct=female   guess=male     name=Willie                        
correct=female   guess=male     name=Xena                          
correct=male     guess=female   name=Adam                          
correct=male     guess=female   name=Aditya                        
correct=male     guess=female   name=Adolpho                       
correct=male     guess=fe

181

#### Last Letter function :

In [8]:
def last_letter(name):
    features = {}
    features["last_letter"] = name[-1].lower()
    return features

In [9]:
train_dataset_last_letter = [(last_letter(n), g) for (n,g) in data_train]
devtest_dataset_last_letter = [(last_letter(n), g) for (n,g) in data_devtest]
test_dataset_last_letter = [(last_letter(n), g) for (n,g) in data_test]
nb2 = nltk.NaiveBayesClassifier.train(train_dataset_last_letter) 
print('Dev-set accuracy is')
print(nltk.classify.accuracy(nb2, devtest_dataset_last_letter))
print('Test accuracy is')
print(nltk.classify.accuracy(nb2, test_dataset_last_letter))

Dev-set accuracy is
0.736
Test accuracy is
0.716


In [10]:
error_analysis(last_letter,nb2,display=True)

Number of Errors:  132
correct=female   guess=male     name=Adelind                       
correct=female   guess=male     name=Allis                         
correct=female   guess=male     name=Annabal                       
correct=female   guess=male     name=Bell                          
correct=female   guess=male     name=Beryl                         
correct=female   guess=male     name=Caron                         
correct=female   guess=male     name=Cathleen                      
correct=female   guess=male     name=Charis                        
correct=female   guess=male     name=Charmain                      
correct=female   guess=male     name=Christin                      
correct=female   guess=male     name=Coralyn                       
correct=female   guess=male     name=Cris                          
correct=female   guess=male     name=Dyan                          
correct=female   guess=male     name=Ellynn                        
correct=female   guess=ma

132

#### Gender Features function from book

In [11]:
def gender_features_from_book(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

In [12]:
train_dataset_gender_features_from_book = [(gender_features_from_book(n), g) for (n,g) in data_train]
devtest_dataset_gender_features_from_book = [(gender_features_from_book(n), g) for (n,g) in data_devtest]
test_dataset_gender_features_from_book = [(gender_features_from_book(n), g) for (n,g) in data_test]
classifier = nltk.NaiveBayesClassifier.train(train_dataset_gender_features_from_book)
nb3 = nltk.NaiveBayesClassifier.train(train_dataset_gender_features_from_book) 
print('Devset accuracy is')
print(nltk.classify.accuracy(nb3, devtest_dataset_gender_features_from_book))
print('Test accuracy is')
print(nltk.classify.accuracy(nb3, test_dataset_gender_features_from_book))

Devset accuracy is
0.772
Test accuracy is
0.734


In [13]:
error_analysis(gender_features_from_book,nb3,display=True)

Number of Errors:  114
correct=female   guess=male     name=Auguste                       
correct=female   guess=male     name=Berte                         
correct=female   guess=male     name=Beryl                         
correct=female   guess=male     name=Bobbie                        
correct=female   guess=male     name=Caron                         
correct=female   guess=male     name=Charis                        
correct=female   guess=male     name=Chery                         
correct=female   guess=male     name=Christin                      
correct=female   guess=male     name=Courtenay                     
correct=female   guess=male     name=Cris                          
correct=female   guess=male     name=Farah                         
correct=female   guess=male     name=Florette                      
correct=female   guess=male     name=Florrie                       
correct=female   guess=male     name=Gates                         
correct=female   guess=ma

114

#### Update Gender Features function :

In [14]:
# define second version of gender features
def gender_features_updated(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in "abcdefghijklmnopqrstuvwxyz":
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    features["suffix2"] = name[-2:].lower()
    features["suffix3"] = name[-3:].lower()
    features["prefix2"] = name[:2].lower()
    features["prefix3"] = name[:3].lower()
    return features

In [15]:
train_dataset_gender_features_updated = [(gender_features_updated(n), g) for (n,g) in data_train]
devtest_dataset_gender_features_updated = [(gender_features_updated(n), g) for (n,g) in data_devtest]
test_dataset_gender_features_updated = [(gender_features_updated(n), g) for (n,g) in data_test]
nb4 = nltk.NaiveBayesClassifier.train(train_dataset_gender_features_updated) 
print('Devset accuracy is')
print(nltk.classify.accuracy(nb4, devtest_dataset_gender_features_updated))
print('Test accuracy is')
print(nltk.classify.accuracy(nb4, test_dataset_gender_features_updated))

Devset accuracy is
0.804
Test accuracy is
0.796


In [16]:
error_analysis(gender_features_updated,nb4,True)

Number of Errors:  98
correct=female   guess=male     name=Ajay                          
correct=female   guess=male     name=Caron                         
correct=female   guess=male     name=Charis                        
correct=female   guess=male     name=Charmain                      
correct=female   guess=male     name=Chery                         
correct=female   guess=male     name=Christin                      
correct=female   guess=male     name=Courtenay                     
correct=female   guess=male     name=Evey                          
correct=female   guess=male     name=Gates                         
correct=female   guess=male     name=Gen                           
correct=female   guess=male     name=Gill                          
correct=female   guess=male     name=Gretel                        
correct=female   guess=male     name=Guenevere                     
correct=female   guess=male     name=Hedy                          
correct=female   guess=mal

98

####  Custom Function :

Create a function to generate new features which are name's last letter, last two letters, the last 3 letters, the first 2 letters,the first 3 letters, and first 4 letters.

In [17]:
def custom_features(name):
        features = {}
        features["suffix1"] = name[-1:].lower()
        features["suffix2"] = name[-2:].lower()
        features["suffix3"] = name[-3:].lower()
        features["prefix2"] = name[:2].lower()
        features["prefix3"] = name[:3].lower()
        features["prefix4"] = name[:4].lower()
        return features


In [18]:
train_dataset_custom_features = [(custom_features(n), g) for (n,g) in data_train]
devtest_dataset_custom_features = [(custom_features(n), g) for (n,g) in data_devtest]
test_dataset_custom_features = [(custom_features(n), g) for (n,g) in data_test]
nb5 = nltk.NaiveBayesClassifier.train(train_dataset_custom_features) 
print('Devset accuracy is')
print(nltk.classify.accuracy(nb4, devtest_dataset_custom_features))
print('Test accuracy is')
print(nltk.classify.accuracy(nb4, test_dataset_custom_features))

Devset accuracy is
0.82
Test accuracy is
0.79


In [19]:
error_analysis(custom_features,nb5,True)

Number of Errors:  82
correct=female   guess=male     name=Ajay                          
correct=female   guess=male     name=Allis                         
correct=female   guess=male     name=Charis                        
correct=female   guess=male     name=Charmain                      
correct=female   guess=male     name=Christin                      
correct=female   guess=male     name=Courtenay                     
correct=female   guess=male     name=Evey                          
correct=female   guess=male     name=Gates                         
correct=female   guess=male     name=Gen                           
correct=female   guess=male     name=Gill                          
correct=female   guess=male     name=Gretel                        
correct=female   guess=male     name=Ines                          
correct=female   guess=male     name=Jaleh                         
correct=female   guess=male     name=Jonis                         
correct=female   guess=mal

82

In [20]:
nb5.show_most_informative_features(10)

Most Informative Features
                 suffix2 = 'na'           female : male   =     87.2 : 1.0
                 suffix2 = 'ia'           female : male   =     80.6 : 1.0
                 suffix2 = 'la'           female : male   =     68.2 : 1.0
                 suffix1 = 'a'            female : male   =     41.7 : 1.0
                 suffix2 = 'ld'             male : female =     35.0 : 1.0
                 suffix2 = 'us'             male : female =     33.7 : 1.0
                 suffix2 = 'sa'           female : male   =     31.0 : 1.0
                 suffix1 = 'k'              male : female =     29.4 : 1.0
                 suffix2 = 'ta'           female : male   =     28.5 : 1.0
                 suffix3 = 'ita'          female : male   =     25.1 : 1.0


In [21]:
error_analysis(first_letter,nb1,display=False)

181

### Summary Display :

### Total Error :

#### Dev-Test  Total Error

In [22]:
total_error_dev_test_df = pd.DataFrame()
total_error_dev_test_df = total_error_dev_test_df.append({
     "First Letter": error_analysis(first_letter,nb1,display=False),
     "Last Letter":  error_analysis(last_letter,nb2,display=False),
     "Gender Features from Book": error_analysis(gender_features_from_book,nb3,display=False),
     "Gender Features Updates" : error_analysis(gender_features_updated,nb4,display=False),
     "Custom Features" : error_analysis(custom_features,nb5,display=False)
      }, ignore_index=True)
total_error_dev_test_df

,Custom Features,First Letter,Gender Features Updates,Gender Features from Book,Last Letter
0,82.0,181.0,98.0,114.0,132.0


#### Test Total Error

In [23]:
total_error_test_df = pd.DataFrame()
total_error_test_df = total_error_test_df.append({
     "First Letter": error_analysis(first_letter,nb1,display=False,dataset='test'),
     "Last Letter":  error_analysis(last_letter,nb2,display=False,dataset='test'),
     "Gender Features from Book": error_analysis(gender_features_from_book,nb3,display=False,dataset='test'),
     "Gender Features Updates" : error_analysis(gender_features_updated,nb4,display=False,dataset='test'),
     "Custom Features" : error_analysis(custom_features,nb5,display=False,dataset='test')
      }, ignore_index=True)
total_error_test_df

,Custom Features,First Letter,Gender Features Updates,Gender Features from Book,Last Letter
0,97.0,165.0,102.0,133.0,142.0


#### Dev-Test Accuracy 

In [24]:
accuracy_dev_test_df = pd.DataFrame()
accuracy_dev_test_df = accuracy_dev_test_df.append({
     "First Letter": nltk.classify.accuracy(nb1, devtest_dataset_first_letter),
     "Last Letter":  nltk.classify.accuracy(nb2, devtest_dataset_last_letter),
     "Gender Features from Book": nltk.classify.accuracy(nb3, devtest_dataset_gender_features_from_book),
     "Gender Features Updates" : nltk.classify.accuracy(nb4, devtest_dataset_gender_features_updated),
     "Custom Features" : nltk.classify.accuracy(nb5, devtest_dataset_custom_features)
      }, ignore_index=True)
accuracy_dev_test_df

,Custom Features,First Letter,Gender Features Updates,Gender Features from Book,Last Letter
0,0.836,0.638,0.804,0.772,0.736


#### Test Accuracy 

In [25]:
accuracy_test_df = pd.DataFrame()
accuracy_test_df = accuracy_test_df.append({
     "First Letter": nltk.classify.accuracy(nb1, test_dataset_first_letter),
     "Last Letter":  nltk.classify.accuracy(nb2, test_dataset_last_letter),
     "Gender Features from Book": nltk.classify.accuracy(nb3, test_dataset_gender_features_from_book),
     "Gender Features Updates" : nltk.classify.accuracy(nb4, test_dataset_gender_features_updated),
     "Custom Features" : nltk.classify.accuracy(nb5, test_dataset_custom_features)
      }, ignore_index=True)
accuracy_test_df

,Custom Features,First Letter,Gender Features Updates,Gender Features from Book,Last Letter
0,0.806,0.67,0.796,0.734,0.716


In [26]:
accuracy_dfs=[accuracy_dev_test_df,accuracy_test_df]
results = pd.concat(accuracy_dfs,axis=0)
results['Test Type'] = ['Accuracy Dev-Test','Accuracy Test']
results

,Custom Features,First Letter,Gender Features Updates,Gender Features from Book,Last Letter,Test Type
0,0.836,0.638,0.804,0.772,0.736,Accuracy Dev- Test
0,0.806,0.670,0.796,0.734,0.716,Accuracy Test


In [27]:
total_error_dfs=[total_error_dev_test_df,total_error_test_df]
results = pd.concat(total_error_dfs,axis=0)
results['Test Type'] = ['Total Error Dev-Test','Total Error Test']
results

,Custom Features,First Letter,Gender Features Updates,Gender Features from Book,Last Letter,Test Type
0,82.0,181.0,98.0,114.0,132.0,Total Error Dev-Test
0,97.0,165.0,102.0,133.0,142.0,Total Error Test


### Conclusion